In [2]:
from src.modules.models import *
from sklearn.model_selection import train_test_split
from src.modules.prediction import *

In [11]:
EXPLIQUEE = "unit_price" #"type" => Classification ou "unit_price" => Regression

In [12]:
if EXPLIQUEE == "type":
    CATEGORICALS = ["cepage", "par_gouts", "service", "country"]
    MODE = "classification"
elif EXPLIQUEE == "unit_price":
    CATEGORICALS = ["cepage", "par_gouts", "service", "country", "type"]
    MODE = "regression"

In [13]:
df = data_model(chemin= "./data/vins.json",
                variable_a_predire= EXPLIQUEE)

In [14]:
df = df.select("capacity", "unit_price","millesime", "cepage", "par_gouts",
          "service", "avg_temp", "conservation_date", "bio", "customer_fav", 
          "is_new", "top_100", "destock", "sulphite_free", "alcohol_volume",
          "country", "bubbles", "wine_note", "nb_reviews", "conservation_time","type")

In [15]:
df = prep_str(df, categorical_cols=CATEGORICALS)

In [16]:
X = df.drop(columns=[EXPLIQUEE])
y = df[EXPLIQUEE]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = Pipeline(
            [
                ("imputation", SimpleImputer()),
                ("echelle", MinMaxScaler()),
                ("entrainement", RandomForestClassifier(max_depth=9, n_estimators=30)),
            ])
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

from sklearn.metrics import mean_squared_error, mean_absolute_error
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))

import matplotlib.pyplot as plt
importances = importances[0:20]
indices = np.argsort(importances)
fig, ax = plt.subplots()
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(X_train.columns)[indices], fontsize = 8)

In [17]:
models = train_model(X_train, y_train, "regression")

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
stockage_result_csv(models, "regression")

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the 

C'est bon ça a marché


In [12]:
import polars as pl
from src.modules.models import *
from src.modules.prediction import * 

In [13]:
VARIABLE = "type"

In [14]:
X_train, X_test, y_train, y_test, data = init(VARIABLE)

In [15]:
masque = data.index.isin(y_test.index)

In [16]:
df = data[masque]

In [17]:
model_rf = random_forest(VARIABLE, "Random Forest")
model_boost = boosting(VARIABLE, "Boosting")
model_ridge = ridge(VARIABLE, "Ridge")
model_knn = knn(VARIABLE, "K Neighbors")
model_mlp = mlp(VARIABLE, "Réseaux de neurones")
model_sv = support_vector(VARIABLE, "Support Vector")

In [18]:
models = [model_rf, model_boost, model_ridge, model_knn, model_mlp, model_sv]

In [19]:
for model in models:
    model.fit(X_train, y_train)

In [20]:
preds_rf = model_rf.predict(X_test).astype(str)
preds_boost = model_boost.predict(X_test).astype(str)
preds_ridge = model_ridge.predict(X_test).astype(str)
preds_knn = model_knn.predict(X_test).astype(str)
preds_mlp = model_mlp.predict(X_test).astype(str)
preds_sv = model_sv.predict(X_test).astype(str)

In [21]:
df = pl.DataFrame(data[masque])
df = df.select(
    pl.col("name"),
    pl.col(VARIABLE),
    random_forest = pl.lit(preds_rf),
    boosting = pl.lit(preds_boost),
    ridge = pl.lit(preds_ridge),
    knn = pl.lit(preds_knn),
    mlp = pl.lit(preds_mlp),
    support_vector = pl.lit(preds_sv)
)

In [22]:
df.write_csv(f"./data/pred_classification.csv", separator=",")

In [3]:
EXPLIQUEE = "unit_price" #type or unit_price

if EXPLIQUEE == "type":
    CATEGORICALS = ["cepage", "par_gouts", "service", "country"]
    MODE = "classification"
elif EXPLIQUEE == "unit_price":
    CATEGORICALS = ["cepage", "par_gouts", "service", "country", "type"]
    MODE = "regression"
    
df = data_model(chemin= "./data/vins.json",
                variable_a_predire= EXPLIQUEE)

In [16]:
df = df.select("capacity", "unit_price","millesime", "cepage", "par_gouts",
          "service", "avg_temp", "conservation_date", "bio", "customer_fav", 
          "is_new", "top_100", "destock", "sulphite_free", "alcohol_volume",
          "country", "bubbles", "wine_note", "nb_reviews", "conservation_time", "cru",
          "type")

df = prep_str(df, categorical_cols=CATEGORICALS)

In [17]:
X = df.drop(columns=[EXPLIQUEE])

In [78]:
import polars as pl
from sklearn.metrics import confusion_matrix, accuracy_score, mean_absolute_error

In [137]:
df = pl.read_csv("C:/Users/guill/OneDrive - Université de Tours/Bureau/M2/Wine-Scraping/data/tables/pred_classification.csv")

In [145]:
y_true = df.select("type")
y_pred = df.select("boosting")

In [147]:
import plotly.express as px
from sklearn.metrics import confusion_matrix

In [156]:
conf_matrix = confusion_matrix(y_true, y_pred)

fig = px.imshow(conf_matrix,
                labels=dict(x="Prédictions", y="Réalité", color="Nombre"),
                x=['Vin Blanc', 'Vin Rosé','Vin Rouge'],
                y=['Vin Blanc', 'Vin Rosé','Vin Rouge'],
                color_continuous_scale='Blues')

fig.update_layout(title='Matrice de Confusion')

fig.show()
